In [1]:
f = open("index.html", "r", encoding="utf-8")
html = f.readlines()
# print(html)
content = [x for x in html if len(x)>1000]
print(len(content))

12


In [2]:
content_str = "".join(content)
content_str = content_str.replace("<br>", "\n")

In [3]:
class Effect:
    def __init__(self, name, index, value, materials: list[str]):
        self.name = name
        self.index = index
        self.value = int(value)
        self.materials = materials # materials name

class Material:
    def __init__(self, name, effects) -> None:
        self.name = name
        self.effects = [int(x) for x in effects] # 4 effects index

In [4]:
import re
split_index = content_str.find("以下是：属性找材料")
content_material = content_str[:split_index]
content_effect = content_str[split_index:]

In [5]:
match_matierals = re.split(r'<span class="color-l?blue-0[34]">\s*([^（]{0,10})（', content_material)[1:] # name, content
match_effects = re.split(r'（(\d+)）\s*(?:(?:<\/span>)?<span class="color-default font-size-23">\s*)?([^￥]{0,14})￥(\d+)', content_effect)[1:] # index, name, value, content
# print(match_matierals)
# print("EndHere\n".join(match_effects))
print(len(match_effects))

220


In [6]:
material_list = []
for i in range(len(match_matierals)//2):
    name = match_matierals[i*2]
    content = match_matierals[i*2+1]
    effects = re.findall(r'（(\d+)）', content)
    material_list.append(Material(name, effects))

In [7]:
effects_list = [0] * 60
for i in range(len(match_effects)//4):
    index = match_effects[i*4]
    name = match_effects[i*4+1]
    value = match_effects[i*4+2]
    content = match_effects[i*4+3]
    materials = re.findall(r'<span class="color-l?blue-0[34]">\s*([^（]{0,10})（', content)
    effects_list[int(index)] = Effect(name, index, value, materials)

In [8]:
with open("Effects.txt", "w") as f:
    for effect in effects_list:
        if isinstance(effect, Effect):
            f.write(f"{effect.name}\n{effect.index}\n{effect.value}\n{effect.materials}\n\n")

with open("Materials.txt", "w") as f:
    for material in material_list:
        f.write(f"{material.name}\n{material.effects}\n\n")

---


In [9]:
class Recipe:
    def __value(self):
        value = 0
        for x in self.effects:
                if isinstance(effects_list[x],Effect):
                    value += effects_list[x].value
                else:
                    print(f"Int Error!{x}")
        return value

    def __init__(self, materials: set[str], effects: set[str]) -> None:
        self.materials = materials
        self.effects = effects
        self.value = self.__value()


Generate Recipes

In [10]:
recipes_file = open("Recipes.csv", "w")
two_list = []
three_list = []

for i in range(len(material_list)):
    for j in range(i+1, len(material_list)):
        m1 = material_list[i]
        m2 = material_list[j]
        e1 = set(m1.effects)
        e2 = set(m2.effects)
        common_effect = e1 & e2
        if len(common_effect) >= 2:
            two_list.append(Recipe(set([m1.name, m2.name]), common_effect))

for i in range(len(two_list)):
    for j in range(i+1, len(two_list)):
        recipe1 = two_list[i]
        recipe2 = two_list[j]
        common_materials = recipe1.materials & recipe2.materials
        if len(common_materials) > 0:
            three_list.append(Recipe(recipe1.materials | recipe2.materials, recipe1.effects | recipe2.effects))


recipes_file.writelines([f"\"{e.materials}\", \"{e.effects}\", {e.value}\n" for e in two_list])
recipes_file.writelines([f"\"{e.materials}\", \"{e.effects}\", {e.value}\n" for e in three_list])

recipes_file.close()